In [37]:
# import json
# import pandas as pd


# with open('prompts.json') as f:
#     data = json.load(f)

# df = pd.DataFrame.from_dict(data, orient='index')

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
csv_path = r"/content/drive/MyDrive/vibviz/vibrationAnnotations-July24th2016.csv"
json_path = r"C:\Users\Domin\OneDrive\HTW\Bachelorarbeit\WAVTactilTransformer\viblib\testShortSignalDetection"

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN = "EOS_TOKEN\n"

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

import os
#Lädt json Datei und gibt Inhalt als string zurück
def readJson(file_name):
  file_path = os.path.join(json_path, file_name + ".json")
  if not os.path.isfile(file_path):
    raise Exception("File", file_path, "does not exists.")
  file = open(file_path,'r')
  content = file.read()
  file.close()
  return content


from datasets import load_dataset
# Datensätze mit fehlender JSON löschen
def delete_if_json_is_missing(dataset):
  keep_indices = []
  for i in range(len(dataset["id"])):
    file_path = os.path.join(json_path, dataset["id"][i] + ".json")

    if not os.path.isfile(file_path):
      print("Warning: File", file_path, "does not exists. Dataset is ignored. Idex:", i)
    else:
      keep_indices.append(i)
  return dataset.select(keep_indices)

def value_to_description(value):
 pass

#Formatiert die Daten für das fine tuning-> Rohdaten aus VibViz Datenbank
def vibviz_formatting_prompts_func(examples):
  instruction = "Generate an vibrotaktil pattern with following characteristics:" #Instruction verknüpt im Modell die Aufgabe mit der Eingabe
  texts = []
  for i in range(len(examples["id"])):
    input = ""
    if examples["sensationTags"][i] is not None:
      input = input + examples["sensationTags"][i] + ","
    if examples["emotionTags"][i] is not None:
      input = input + examples["emotionTags"][i] + ","
    if examples["metaphors"][i] is not None:
      input = input + examples["metaphors"][i] + ","
    if examples["usageExamples"][i] is not None:
      input = input + examples["usageExamples"][i]
    output = readJson(examples["id"][i])
    if output is not None:
      text = alpaca_prompt.format(instruction,input,output) + EOS_TOKEN
      texts.append(text)
  return {"text" : texts}

def vibviz_formatting_natural_prompts_func(examples):
  input = ""
  texts = []
  for i in range(len(examples["id"])):
    instruction = examples["1"][i]
    output = readJson(examples["id"][i])
    text = alpaca_prompt.format(instruction,input,output) + EOS_TOKEN
    texts.append(text)
  return {"text" : texts}

  
fromCSV = False
fromJSON = not fromCSV

if fromCSV:

    #Datem mit fehlender json aussortieren
    #dataset = delete_if_json_is_missing(dataset)

    #dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
    #Daten formatieren "mapen" für das Training
    dataset = dataset.map(vibviz_formatting_prompts_func, batched = True,)

if fromJSON:
    #Daten laden
  import json
  import pandas as pd
  import datasets
  with open('prompts.json') as f:
      data = json.load(f)
  
  df = pd.DataFrame.from_dict(data, orient='index')
  
  datasets.Dataset.from_pandas(df)

  dataset = datasets.Dataset.from_pandas(df, split="train").rename_column("__index_level_0__","id")

  
  dataset = delete_if_json_is_missing(dataset)
  #Daten formatieren "mapen" für das Training
  dataset = dataset.map(vibviz_formatting_natural_prompts_func, batched = True,)

# Entimmt aus den Trainingsdaten zufällige Testdatem
if True:
  dataset = dataset.train_test_split(test_size = 0.03, shuffle = True, seed = 3407)
dataset


Map: 100%|██████████| 119/119 [00:00<00:00, 1183.26 examples/s]


DatasetDict({
    train: Dataset({
        features: ['0', '1', 'id', 'text'],
        num_rows: 115
    })
    test: Dataset({
        features: ['0', '1', 'id', 'text'],
        num_rows: 4
    })
})